# Scrape menus data

In [14]:
import pandas as pd

In [15]:
def detect_text_uri(path):
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    # Open local image
    import io
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image(content=content)
    # image = vision.Image()
    # image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
    # print('Texts:')
    
    text_list = []
    bounding_all = []
    left_up_x = []
    left_up_y = []
    left_bot_x = []
    left_bot_y = []
    right_up_x = []
    right_up_y = []
    right_bot_x = []
    right_bot_y = []

    for text in texts[1:]:
        # print('\n"{}"'.format(text.description))
        new_text = '''{}'''.format(text.description)
        text_list.append(new_text)
        

        vertices = [tuple((vertex.x, vertex.y)) for vertex in text.bounding_poly.vertices]

        # print('bounds: {}'.format(','.join(vertices)))
        new_bounding = vertices
        bounding_all.append(new_bounding)
        left_up_x.append(vertices[0][0])
        left_up_y.append(vertices[0][1])
        left_bot_x.append(vertices[1][0])
        left_bot_y.append(vertices[1][1])
        right_up_x.append(vertices[2][0])
        right_up_y.append(vertices[2][1])
        right_bot_x.append(vertices[3][0])
        right_bot_y.append(vertices[3][1])

        
    menu_df = pd.DataFrame({
        'text': text_list,
        'bounding_all': bounding_all,
        'left_up_x': left_up_x,
        'left_up_y': left_up_y,
        'left_bot_x': left_bot_x,
        'left_bot_y': left_bot_y,
        'right_up_x': right_up_x,
        'right_up_y': right_up_y,
        'right_bot_x': right_bot_x,
        'right_bot_y': right_bot_y,
    })

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return menu_df

In [16]:
!pwd

/Users/MacBookPro/code/nganguyenny/menu_me/notebooks


In [20]:
sample_df = detect_text_uri('../raw_data/all_menus/english_menu_6.jpg')

In [22]:
sample_df

,text,bounding_all,left_up_x,left_up_y,left_bot_x,left_bot_y,right_up_x,right_up_y,right_bot_x,right_bot_y
0,Salads,"[(160, 9), (239, 11), (238, 39), (159, 37)]",160,9,239,11,238,39,159,37
1,ICEBERG,"[(8, 70), (71, 70), (71, 80), (8, 80)]",8,70,71,70,71,80,8,80
2,WEDGE,"[(76, 70), (131, 70), (131, 80), (76, 80)]",76,70,131,70,131,80,76,80
3,avocado,"[(7, 81), (48, 81), (48, 96), (7, 96)]",7,81,48,81,48,96,7,96
4,",","[(48, 81), (50, 81), (50, 95), (48, 95)]",48,81,50,81,50,95,48,95
...,...,...,...,...,...,...,...,...,...,...
186,steak,"[(271, 558), (301, 558), (301, 573), (271, 573)]",271,558,301,558,301,573,271,573
187,(,"[(306, 558), (312, 558), (312, 573), (306, 573)]",306,558,312,558,312,573,306,573
188,$,"[(311, 558), (318, 558), (318, 573), (311, 573)]",311,558,318,558,318,573,311,573
189,12,"[(321, 558), (337, 558), (337, 573), (321, 573)]",321,558,337,558,337,573,321,573


In [27]:
sample_df['height_left'] = abs(sample_df['left_up_x'] - sample_df['left_bot_x'])

In [29]:
sample_df['height_right'] = abs(sample_df['right_up_x'] - sample_df['right_bot_x'])
sample_df

,text,bounding_all,left_up_x,left_up_y,left_bot_x,left_bot_y,right_up_x,right_up_y,right_bot_x,right_bot_y,height_left,height_right
0,Salads,"[(160, 9), (239, 11), (238, 39), (159, 37)]",160,9,239,11,238,39,159,37,79,79
1,ICEBERG,"[(8, 70), (71, 70), (71, 80), (8, 80)]",8,70,71,70,71,80,8,80,63,63
2,WEDGE,"[(76, 70), (131, 70), (131, 80), (76, 80)]",76,70,131,70,131,80,76,80,55,55
3,avocado,"[(7, 81), (48, 81), (48, 96), (7, 96)]",7,81,48,81,48,96,7,96,41,41
4,",","[(48, 81), (50, 81), (50, 95), (48, 95)]",48,81,50,81,50,95,48,95,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...
186,steak,"[(271, 558), (301, 558), (301, 573), (271, 573)]",271,558,301,558,301,573,271,573,30,30
187,(,"[(306, 558), (312, 558), (312, 573), (306, 573)]",306,558,312,558,312,573,306,573,6,6
188,$,"[(311, 558), (318, 558), (318, 573), (311, 573)]",311,558,318,558,318,573,311,573,7,7
189,12,"[(321, 558), (337, 558), (337, 573), (321, 573)]",321,558,337,558,337,573,321,573,16,16


In [30]:
sample_df.to_csv('../csvs/english_menu_6.csv')

In [145]:
sample_df['height_right'].sort_values().value_counts()

2      20
3      11
4       7
29      7
30      7
39      7
26      5
47      5
42      5
41      5
31      5
16      4
20      4
34      4
15      4
37      4
40      4
6       4
38      4
33      3
27      3
55      3
28      3
1       3
12      3
11      3
23      3
21      3
5       3
18      3
17      3
48      2
7       2
45      2
63      2
64      2
25      2
53      2
68      2
36      2
32      2
67      1
60      1
72      1
59      1
79      1
54      1
46      1
51      1
50      1
49      1
24      1
44      1
43      1
8       1
10      1
13      1
14      1
35      1
113     1
Name: height_right, dtype: int64